In [2]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

api_key_4square = os.getenv('FOURSQUARE_KEY')
api_key_yelp = os.getenv('YELP_KEY')

In [5]:
bikes_df =pd.read_csv(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\bikes_df.csv",sep=',')
bikes_df.head()

,id,name,latitude,longitude,free_bikes,empty_slots,uid,station_number,bike_ids
0,066c99293af108ece27d9b0436c30cc4,Riverside Museum,55.865683,-4.305367,4,2,19738176,8416,"['81103', '46982', '46665', '829983']"
1,0a106cbc148d5a0c2535b51c1dbe3b4d,Parkhead,55.846524,-4.197475,4,4,150125231,8470,"['46871', '828262', '46720', '46493']"
2,0cc8c09950e1435ee7782478ed292fef,Tollcross International Swimming Centre,55.844250,-4.176167,4,2,55599921,8200,"['840033', '46734', '46515', '828134']"
3,0e94d2ad012bff0cf23497963ff5fd77,Shields Road,55.845807,-4.275232,1,7,7145152,8403,['81175']
4,101c6cd7749f373507e9444f5222b7f2,Shawlands Shopping Centre (Kilmarnock Road),55.829057,-4.282675,4,4,3812605,8453,"['826907', '46550', '828983', '828140']"


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
# 4square api prep function
def places_4square(latitude, longitude, radius, categories,limit, api_key):
    params = {}
    ll = str(latitude)+","+str(longitude)
    params["ll"] = ll
    params['radius'] = radius
    categories = ','.join(str(x)for x in categories)
    params["categories"] = categories
    params["limit"] = limit

    url = "https://api.foursquare.com/v3/places/search"
    headers = {"accept": "application/json","Authorization": api_key}
    response = requests.get(url, params=params, headers=headers)
    return response

In [4]:
# # Getting and saving response data as a local json file
# category_list=[16033,13003]
# foursquare_places_json = places_4square(bikes_df['latitude'][0],bikes_df['longitude'][0],1000,category_list,50,api_key_4square).json()
# with open('data/foursquare_places_data.json','w') as f:
#     json.dump(foursquare_places_json,f)
# foursquare_places_json

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [5]:
# pulling data from json into dataframe
# lists for temp busckets
id = []
categories = []
distance = []
name = []

temp = []

#loading json file
with open(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\foursquare_places_data.json",'r') as f:
    data= json.load(f)
    
# run through all results for each place info
for result in range(len(data['results'])):
    id.append(data['results'][result]['fsq_id'])
    # run thorugh all categories for each place
    for i in range(len(data['results'][result]['categories'])):
        temp.append(data['results'][result]['categories'][i]['short_name'])
    # add categories to list from temp list
    categories.append(temp)
    # clear temp list
    temp = []
    distance.append(data['results'][result]['distance'])
    name.append(data['results'][result]['name'])

Put your parsed results into a DataFrame

In [6]:
# dataframe creation
foursquare_places_df = pd.DataFrame({'id':id, 'name':name,"categories":categories,'distance':distance})
foursquare_places_df.head()

,id,name,categories,distance
0,4e1c4c481fc714dc5c3143ed,BrewDog Glasgow,"[Beer Bar, Burgers]",884
1,4b76fff5f964a52018732ee3,The Lismore,[Whisky Bar],636
2,50d20bc9e4b029fc6804c7a9,The Sparkle Horse,"[Bar, Restaurant]",705
3,4d0263ce3fd4224b72abb677,Three Judges,[Pub],632
4,57cf101c498e00f91f9e64cc,Bag O Nails,"[Beer Bar, Pub, Restaurant]",636


In [7]:
# save dataframe as CSV to access in other notebooks
foursquare_places_df.to_csv(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\foursquare_places_df.csv',sep =',',index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [12]:
# yelp api prep function
def places_yelp(latitude, longitude, radius, categories,limit, api_key):
    params = {}
    params["longitude"] = longitude
    params["latitude"] = latitude
    params['radius'] = radius
    params["categories"] = categories
    params["limit"] = limit

    url = f"https://api.yelp.com/v3/businesses/search?"
    headers = {"accept": "application/json","Authorization": f'bearer {api_key}'}
    response = requests.get(url, params=params, headers=headers)
    return response

In [53]:
# # Getting and saving response data as a local json file
# category_list=['bars', 'dog_parks']
# yelp_places_json = places_yelp(bikes_df['latitude'][0],bikes_df['longitude'][0],1000,category_list,10,api_key_yelp).json()
# with open('data/yelp_places_data.json','w') as f:
#     json.dump(yelp_places_json,f)
# yelp_places_json

{'businesses': [{'id': 'qjjmYSR3CVsJlr75vyIEBg',
   'alias': 'brewdog-glasgow-glasgow',
   'name': 'BrewDog Glasgow',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/dfxLIhNGx8-EEEgtf56L2g/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/brewdog-glasgow-glasgow?adjust_creative=VTfKcdmaRmcHX2FPNLMuYA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=VTfKcdmaRmcHX2FPNLMuYA',
   'review_count': 109,
   'categories': [{'alias': 'bars', 'title': 'Bars'}],
   'rating': 4.3,
   'coordinates': {'latitude': 55.8677022872916,
    'longitude': -4.29187774658203},
   'transactions': [],
   'price': '££',
   'location': {'address1': '1397 Argyle Street',
    'address2': '',
    'address3': '',
    'city': 'Glasgow',
    'zip_code': 'G3 8AN',
    'country': 'GB',
    'state': 'GLG',
    'display_address': ['1397 Argyle Street',
     'Glasgow G3 8AN',
     'United Kingdom']},
   'phone': '+441413347175',
   'display_phone': '+44 141 334 7175',
   'distan

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [54]:
# pulling data from json into dataframe
# lists for temp busckets
id = []
categories = []
distance = []
name = []


#loading json file
with open(r"B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\yelp_places_data.json",'r') as f:
    data= json.load(f)
    
# run through all results for each place info
for result in range(len(data['businesses'])):
    id.append(data['businesses'][result]['id'])
    categories.append(data['businesses'][result]['categories'][0]['alias'])
    distance.append(data['businesses'][result]['distance'])
    name.append(data['businesses'][result]['name'])

Put your parsed results into a DataFrame

In [55]:
# dataframe creation
yelp_places_df = pd.DataFrame({'id':id, 'name':name,"categories":categories,'distance':distance})
yelp_places_df

,id,name,categories,distance
0,qjjmYSR3CVsJlr75vyIEBg,BrewDog Glasgow,bars,871.079804
1,PkLiavkwqsGzE5D2tqfsyg,The Sparkle Horse,pubs,689.080576
2,58dUYvYmmE2h19Q3qITHaA,Brel,belgian,1237.724147
3,_S0eZCB0-qNCAMTrlAIqCw,6 Degrees North,pubs,854.560573
4,Gi2Fs1Ve7lyJAXWKeQkthg,Three Judges,pubs,637.835306
5,c2fwMshDF0wtCKGLe3IGtw,Dukes,pubs,838.518390
6,NTklMMCjRBKMklSwl4YvXg,Tennents Bar,pubs,1141.982966
7,V3ma6UVAs0FWx9eHHSnnMQ,Innis & Gunn Beer Kitchen,beerbar,1240.339614
8,dMdrLM-SMYuiGzzo6Si2Cw,Firebird,italian,937.443536
9,Se9BxMKBLsjTweiYc8-xJg,Vodka Wodka,pubs,1264.736501


In [56]:
# save dataframe as CSV to access in other notebooks
yelp_places_df.to_csv(r'B:\LHL\StatsModeling_Project\LHL-statsModeling-priject\data\yelp_places_df.csv',sep =',',index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating